#### One time runs

In [1]:
! pip install chromadb sentence-transformers langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00:00:01


In [2]:
! pip install protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.2
    Uninstalling protobuf-4.25.2:
      Successfully uninstalled protobuf-4.25.2


In [ ]:
#Downloading the data from gdrive
! pip install gdown
!gdown 'https://drive.google.com/uc?id=1pgvce7x6b9BTWNqHZqN2Z4TkR0hkJp4q&export=download'

### Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import os
from langchain.vectorstores import Chroma
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel

/tmp/ipykernel_3956573/1563873738.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [15]:
os.environ["OPENAI_API_KEY"] = ""

### Testing

In [2]:
testdf = pd.read_pickle('testdf')
testdf.head()

,text,source,content
0,"WM .GARDNER & SONS , Engineers , GLOUCESTER I ...",ID: http://data.odeuropa.eu/smell/00000d38-58e...,wm gardner son engineer gloucester lii chemist...
1,) { ac ) Nerves of the septum nasi .{ aca ) Ol...,ID: http://data.odeuropa.eu/smell/00001f4d-842...,ac nerve septum nasi aca olfactory nerve nn ol...
2,5 .Entirely volatilized by heat ; burns in air...,ID: http://data.odeuropa.eu/smell/00002376-b99...,5 entirely volatilized heat burn air blue flam...
3,There are instruments for measuring the sense ...,ID: http://data.odeuropa.eu/smell/0000263b-7fd...,instrument measuring sense smell olfactometer ...
4,"THE POWDER , when exposed to the air , tut mor...",ID: http://data.odeuropa.eu/smell/00002ae9-624...,powder exposed air tut rapidly damped water wh...


In [3]:
loader2 = DataFrameLoader(testdf, page_content_column="content")
docs2 = loader2.load()

In [4]:
#random document
docs2[np.random.randint(low=1, high=1000)]

Document(page_content='labyrinth miserable tenement allowed grow golden square burlington garden even one well acquainted region get entangled lane broad street great pulteney street find puzzled get breathe many strange odour walk select company fact within stone throw finest street london territory look properly belonged whitechapel wapping book history london map illustration supplement written william john loftie 1883 envokes emotion doubt', metadata={'text': "A labyrinth of miserable tenements had been allowed to grow up between Golden Square and Burlington Gardens .Even now , any one not very well acquainted with the region , who gets entangled in the lanes about Broad Street or Great Pulteney Street will find himself puzzled how to get out again , and will have to breathe many strange odours , and walk in not very select company .There is , in fact , within a stone ' s throw of the finest street in London a territory which looks as if it properly belonged to Whitechapel or Wappi

In [5]:
model_kwargs = {'device':'cuda'}

In [6]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs=model_kwargs)

C:\Users\bipna\anaconda3\envs\odeuropa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# load it into Chroma
test_vectordb = Chroma.from_documents(docs2, embedding_function,persist_directory='test_vectordb')

In [ ]:
query = "from the book A further account of the proceedings of the Association of Fellows of the Royal College of Surgeons of England"
retrived_docs = test_vectordb.similarity_search_with_score(query,k=3)

# print results
for retrived_doc in retrived_docs:
  print(retrived_doc)

### Vector DBs

Vector db <b>Naming scheme</b> : db_[model]_[sw/nsw]_[ck/nck]<br>
swr/swn: stopword removed/stopword not removed<br>
ck/nck: chunked/not chunked

#### DF management
create df as per requirements of vector dbs

In [2]:
embedding_df = pd.read_pickle('../data/clean_embedding_df.pkl')
embedding_df

,text,source,content
0,"WM .GARDNER & SONS , Engineers , GLOUCESTER I ...",ID: http://data.odeuropa.eu/smell/00000d38-58e...,wm gardner son engineer gloucester lii chemist...
1,) { ac ) Nerves of the septum nasi .{ aca ) Ol...,ID: http://data.odeuropa.eu/smell/00001f4d-842...,ac nerve septum nasi aca olfactory nerve nn ol...
2,5 .Entirely volatilized by heat ; burns in air...,ID: http://data.odeuropa.eu/smell/00002376-b99...,5 entirely volatilized heat burn air blue flam...
3,There are instruments for measuring the sense ...,ID: http://data.odeuropa.eu/smell/0000263b-7fd...,instrument measuring sense smell olfactometer ...
4,"THE POWDER , when exposed to the air , tut mor...",ID: http://data.odeuropa.eu/smell/00002ae9-624...,powder exposed air tut rapidly damped water wh...
...,...,...,...
1452082,The effect produced on the bell wires was beau...,ID: http://data.odeuropa.eu/smell/ffffdd4d-cf7...,effect produced bell wire wa beautiful beyond ...
1452083,"The common practice of burning scented paper ,...",ID: http://data.odeuropa.eu/smell/ffffde20-a5b...,common practice burning scented paper pastille...
1452084,â   In all ear and throat diseases cleanline...,ID: http://data.odeuropa.eu/smell/ffffe215-eff...,ear throat disease cleanliness nose assist cur...
1452085,Mr .Prosper himself was the stupidest ass ! At...,ID: http://data.odeuropa.eu/smell/ffffe6f7-47d...,mr prosper wa stupidest welwyn people smelled ...


#### MiniLm using stopword removed and not chunked

In [16]:
loader = DataFrameLoader(embedding_df, page_content_column="content")
docs = loader.load()

In [27]:
#random document
docs[np.random.randint(low=1, high=1000)]

Document(page_content='crust ice formed juice poured another vessel repeating process several time remaining juice informed ha concentrated eight time original strength kept without suffering material change several year ice first formed wholly void acidity subsequent congelation become imbued acid whytt found juice lemon allay hysterical palpitation heart various medicine experienced ineffectual juice orange taken quantity four six ounce day ha sometimes found remedy jaundice exterior rind lemon grateful aromatic bitter le hot orange peel yield distillation le quantity oil oil extremely light almost colourless smell nearly agreeable fresh peel frequently employed per fume generally brought u southern part europe name essence lemon lemon peel f see georgii l book report 1957 written ely rural district council 1957 ha smell source oil', metadata={'text': 'After a crust of ice is formed , the juice is poured into another vessel ; and by repeating this process several times , the remainin

In [20]:
# Check if GPU (CUDA) is available
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs:", torch.cuda.device_count())
    current_device = torch.cuda.current_device()
    print("Current GPU:", current_device)
    print("GPU Name:", torch.cuda.get_device_name(current_device))
else:
    print("CUDA is not available. Using CPU.")
    current_device = torch.device("cpu")

# Set CUDA_VISIBLE_DEVICES to specify the GPU
torch.cuda.set_device(3)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

CUDA is available. Number of GPUs: 4
Current GPU: 3
GPU Name: NVIDIA GeForce RTX 3090


In [21]:
model_kwargs = {'device':'cuda'}

In [23]:
# create the open-source embedding function
MiniLM = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs=model_kwargs)

In [25]:
#load it into Chroma
#db_miniLM_swr_nck = Chroma.from_documents(docs,MiniLM,persist_directory='../data/db_miniLM_swr_nck')

In [8]:
db_miniLM_swr_nck = Chroma(persist_directory="../data/db_miniLM_swr_nck", embedding_function=MiniLM)

In [26]:
query = "What was the tobacco smelled like in 1800s "
retrived_docs = db_miniLM_swr_nck.similarity_search_with_score(query,k=5)

# print results
for retrived_doc in retrived_docs:
  print(retrived_doc)

(Document(page_content='smell well never thought ah scent tobacco would worth great deal u investigation never smoke think observed smell tobacco book strand magazine 1893 ha smell source tobacco', metadata={'source': 'ID: http://data.odeuropa.eu/smell/553f344b-2e59-5ade-a648-011a15bbdce1 Source: http://data.odeuropa.eu/source/f2562bfd-1b0a-574c-87cc-27288255440d', 'text': '" " No smell ? " " Well , we never thought of that ." " Ah , a scent of tobacco would have been worth a great deal to us in such an investigation ." " I never smoke myself , so I think I should have observed it if there had been any smell of tobacco . from the book The Strand Magazine in 1893 has smell source Tobacco.'}), 0.5634164810180664)
(Document(page_content='odeur de tabac f smell bacco gesellschaft f book nouveau dictionnaire franaishollandaisallemandanglais 1849 ha smell source tobacco', metadata={'source': 'ID: http://data.odeuropa.eu/smell/76496e67-89b4-5bd9-b963-fb8713c4ecb5 Source: http://data.odeuropa.

### bge-small-en using stopword removed and not chunked
https://huggingface.co/BAAI/bge-small-en-v1.5

In [3]:
embedding_df

,text,source,content
0,"WM .GARDNER & SONS , Engineers , GLOUCESTER I ...",ID: http://data.odeuropa.eu/smell/00000d38-58e...,wm gardner son engineer gloucester lii chemist...
1,) { ac ) Nerves of the septum nasi .{ aca ) Ol...,ID: http://data.odeuropa.eu/smell/00001f4d-842...,ac nerve septum nasi aca olfactory nerve nn ol...
2,5 .Entirely volatilized by heat ; burns in air...,ID: http://data.odeuropa.eu/smell/00002376-b99...,5 entirely volatilized heat burn air blue flam...
3,There are instruments for measuring the sense ...,ID: http://data.odeuropa.eu/smell/0000263b-7fd...,instrument measuring sense smell olfactometer ...
4,"THE POWDER , when exposed to the air , tut mor...",ID: http://data.odeuropa.eu/smell/00002ae9-624...,powder exposed air tut rapidly damped water wh...
...,...,...,...
1452082,The effect produced on the bell wires was beau...,ID: http://data.odeuropa.eu/smell/ffffdd4d-cf7...,effect produced bell wire wa beautiful beyond ...
1452083,"The common practice of burning scented paper ,...",ID: http://data.odeuropa.eu/smell/ffffde20-a5b...,common practice burning scented paper pastille...
1452084,â   In all ear and throat diseases cleanline...,ID: http://data.odeuropa.eu/smell/ffffe215-eff...,ear throat disease cleanliness nose assist cur...
1452085,Mr .Prosper himself was the stupidest ass ! At...,ID: http://data.odeuropa.eu/smell/ffffe6f7-47d...,mr prosper wa stupidest welwyn people smelled ...


In [4]:
loader = DataFrameLoader(embedding_df, page_content_column="content")
docs = loader.load()

In [5]:
#random document
docs[np.random.randint(low=1, high=1000)]

Document(page_content='among number calculated several thou sand affected carbuncle glanders farcy stench abominable drain solid fluid preserved special purpose yet workman family stout healthy long lived must owing fact foul emanation people constantly inhale act preventive principle poison glanders hence poison might used remedial agent destructive effect book homopathic theory practice electronic resource homopathic treatment surgical disease designed student practitioner medicine guide intelligent public generally written charles j hempel 1872', metadata={'text': 'Among this number , it is calculated that several thou - sand are affected with carbuncle , glanders or farcy .The stench is abominable , for there is no drain ; all the solids and fluids are preserved for special purposes ; and yet all the workmen and their families are stout , healthy , and long - lived .This must be owing to the fact that the foul emanations , which these people constantly inhale , act as a preventive 

In [10]:
# Set CUDA_VISIBLE_DEVICES to specify the GPU
torch.cuda.set_device(3)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# Check if GPU (CUDA) is available
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs:", torch.cuda.device_count())
    current_device = torch.cuda.current_device()
    print("Current GPU:", current_device)
    print("GPU Name:", torch.cuda.get_device_name(current_device))
else:
    print("CUDA is not available. Using CPU.")
    current_device = torch.device("cpu")

CUDA is available. Number of GPUs: 4
Current GPU: 3
GPU Name: NVIDIA GeForce RTX 3090


In [11]:
model_kwargs = {'device':'cuda'}

In [12]:
# create the open-source embedding function
bge = SentenceTransformerEmbeddings(model_name='BAAI/bge-small-en-v1.5', model_kwargs=model_kwargs)

In [13]:
#load it into Chroma
db_bge_swr_nck2 = Chroma.from_documents(docs,bge,persist_directory='../data/db_bge_swr_nck2')

In [32]:
#db_bge_swr_nck = Chroma(persist_directory="../data/db_bge_swr_nck", embedding_function=bge)

In [16]:
query = "What was the tobacco smelled like in 1800s "
retrived_docs = db_bge_swr_nck2.similarity_search_with_score(query,k=5)

# print results
for retrived_doc in retrived_docs:
  print(retrived_doc)

(Document(page_content='tobacco ruminant quid point view flavour savour aroma tobacco shown positively resultant twenty four chemical ingredient naturally enter composition trouble reader learned catalogue weed con stituents besieging army wa plentifully supplied thing came thither sea meuse rich country brabant siege wa long continued brabanters many laro e machine threw town stone weight much damao e wherever fell addition flung town dead carrion tlie army empoison inhabitant stink wa sorely felt weather wa hot air calm carry villanous smell book smoker guide philosopher friend smoke smoke whole whats tobacco historical botanical manufactural anecdotal social medical c written andrew steinmetz 1881 ha smell source tobacco', metadata={'source': 'ID: http://data.odeuropa.eu/smell/506e488f-fe48-5995-a7f6-eb5c1c01a5ab Source: http://data.odeuropa.eu/source/6e2dbb08-4c39-5139-892a-fbbe5da00cee', 'text': 'OF TOBACCO IN THE RUMINANT OR QUID POINT OF VIEW .The flavour , savour , and aroma of

## LLM

In [23]:
os.environ["OPENAI_API_KEY"] = ""

In [38]:
summary_template = """Generate a standalone question which is based on the new question plus the chat history. Just create the standalone question without commentary.

Chat History:
{chat_history}
New question: {question}
Standalone question:"""

summary_template = """Rewrite the question to make it optimal for retrieval of context from a vector database. Return only the new, rewritten question
Question: {question}
"""

summary_prompt = PromptTemplate.from_template(summary_template)

question_template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
question_prompt = ChatPromptTemplate.from_template(question_template)

In [25]:
retriever = clean_vectordb.as_retriever(search_kwargs={"k": 5})
model = ChatOpenAI()
output_parser = StrOutputParser()

summary_chain = summary_prompt | model | output_parser

retrieve_context = RunnableParallel({'context': retriever, 'question': RunnablePassthrough()})

standalone_chain = question_prompt | model | output_parser

In [40]:
def chat(chat_history, question):
  if len(chat_history) == 0:
    standalone_question = question
  else:
    summary_chain_input = {'chat_history': chat_history, 'question': question}
    standalone_question = summary_chain.invoke(summary_chain_input)

  retrieval = retrieve_context.invoke(standalone_question)
  new_context = retrieval["context"]
  # print(new_context)
  # print(type(new_context))

  source_list = [(docum.metadata['source']) for docum in new_context]
  # for docum in new_context:
  #   print(docum)
        # print(source_list)
  # print(type(source_list))


  text_list = [("Text "+str(new_context.index(docum)+1) + ": " + docum.metadata['text']) for docum in new_context]
  print(text_list)
  # print(type(text_list))


  #Can make decision how many docs to use for relevant answer. Need to check retrieval (because DB is not working)
  answer = standalone_chain.invoke({"context": str(text_list), "question": standalone_question})
  chat_history.append({'question': standalone_question, 'answer': answer})
  if len(chat_history) > 3:
    chat_history = chat_history[1:]

  return source_list, chat_history, answer, standalone_question

In [47]:
sources,chathistory,answer, question = chat([""],'Describe the scent of flower')

['Text 1: â \x80 \x99 Its sweet - scented blossoms , whose smell resembles that of the Almond , are striped - with white and rose colours , and with its leaves of a delicate green , trail along the meadow , or around the trunks of trees , and sometimes support themselves by clinging to the corn .The fragrance of these flowers may be enjoyed by all during the summer and autumnal months , with the many others to be found in every spot where the grass is growing and the wild - flowers budding .Other dainty kinds will be found under Ipomcua . from the book Sweet-scented flowers and fragrant leaves : interesting associations gathered from many sources, with notes on their history and utility written by Donald McDonald in 1895 has smell source Flower.', 'Text 2: The scent which is liberated from the flowers of the Tree of Heaven ( Ailanthus ) , of the Horse - chestnut ( AEsculus Hippocastanum ) , of the Flowering Ash ( Fraxinus Ornus ) , and of the Evening Primrose ( ( Enothera ) , resembles

In [48]:
print(sources,chathistory,answer, question)

['ID: http://data.odeuropa.eu/smell/fe82606b-9dd2-5861-bf15-7b7621b72247 Source: http://data.odeuropa.eu/source/e9fb1425-e8f0-5209-a531-4213bc540732', 'ID: http://data.odeuropa.eu/smell/ef1c4900-fc24-5145-a019-5e74cfc70466 Source: http://data.odeuropa.eu/source/5d86299a-f718-570b-a1d6-5171545c4b1c', 'ID: http://data.odeuropa.eu/smell/1affd04b-67e2-516d-af5e-70c3b37c41d9 Source: http://data.odeuropa.eu/source/733bcb91-61f6-5875-846a-f3549ce52672', 'ID: http://data.odeuropa.eu/smell/3b0d07c8-21fc-5d70-a7d7-e596d3c651bb Source: http://data.odeuropa.eu/source/d50a6c56-4ad9-5923-8c24-db25fbc191a0', 'ID: http://data.odeuropa.eu/smell/e1f0af2f-34e0-58ba-b331-b0bf7baa73ac Source: http://data.odeuropa.eu/source/19e62cba-cab8-5b71-a8b5-43687eed62d2'] ['', {'question': 'What is the scent of a flower like?', 'answer': 'Based on the given context, the scent of a flower can be described as sweet-scented, resembling that of the Almond, striped with white and rose colors, resembling that of Crataegus,